# Fine-tuning

## 安装

注意：若要使用 FlagEmbedding 对 BGE 模型进行微调，需安装带有 finetune 依赖项的包：

```powershell
pip install -U FlagEmbedding[finetune]
```

macOS环境：

```zsh
pip install -U "FlagEmbedding[finetune]"
pip install deepspeed
```

## 微调

以下是微调时可用的参数：

### 模型相关参数：
- `model_name_or_path`：用于初始化的模型检查点路径或名称。
- `config_name`：若与 `model_name_or_path` 不同，则指定预训练配置的名称或路径。
- `tokenizer_name`：若与 `model_name_or_path` 不同，则指定预训练分词器的名称或路径。
- `cache_dir`：指定用于存储从 Hugging Face 下载的预训练模型的缓存目录。
- `trust_remote_code`：是否信任远程代码（用于加载自定义模型代码）。
- `token`：访问模型时使用的 Hugging Face 访问令牌（用于私有模型）。

### 数据相关参数：
- `train_data`：一个或多个训练数据路径。训练数据中必须包含 `query: str`、`pos: List[str]`（正例）和 `neg: List[str]`（负例）字段。该参数支持多个值。
- `cache_path`：指定缓存数据的存储路径。
- `train_group_size`：（无详细说明）
- `query_max_len`：查询（query）经分词后的最大输入序列长度，超出部分将被截断。
- `passage_max_len`：段落（passage）经分词后的最大输入序列长度，超出部分将被截断。
- `pad_to_multiple_of`：若设置，会将序列填充至该值的整数倍。
- `max_example_num_per_dataset`：每个数据集最多使用的样本数量。
- `query_instruction_for_retrieval`：用于检索任务的查询指令文本。
- `query_instruction_format`：查询指令的格式模板（如 `"{} {}"`）。
- `knowledge_distillation`：若训练数据中包含 `pos_scores: List[float]` 和 `neg_scores: List[float]` 字段，则启用知识蒸馏。
- `passage_instruction_for_retrieval`：用于检索任务的段落指令文本。
- `passage_instruction_format`：段落指令的格式模板。
- `shuffle_ratio`：文本打乱的比例。
- `same_dataset_within_batch`：是否保证同一批次（batch）中的所有样本均来自同一数据集。
- `small_threshold`：小数据集的阈值。同一目录下所有小于该阈值的数据集将被合并为一个数据集。
- `drop_threshold`：合并后小数据集的丢弃阈值。若合并后样本数小于此值，则丢弃该数据集。

### 其他额外参数：
- `negatives_cross_device`：是否在不同设备（如多 GPU）之间共享负例。
- `temperature`：用于计算相似度分数的温度参数（通常用于对比学习中的 softmax）。
- `fix_position_embedding`：是否冻结位置编码（position embedding）的参数。
- `sentence_pooling_method`：句子嵌入的池化方法。可选值：`cls`、`mean`、`last_token`。默认：`cls`。
- `normalize_embeddings`：是否对输出的嵌入向量进行 L2 归一化。
- `sub_batch_size`：训练时的子批次大小（用于梯度累积等场景）。
- `kd_loss_type`：知识蒸馏所用的损失函数类型。可选值：`kl_div`、`m3_kd_loss`。默认：`kl_div`。

MacOS（Apple Silicon）环境参数：

```bash
torchrun --nproc_per_node 2 \
	-m FlagEmbedding.finetune.embedder.encoder_only.base \
	--model_name_or_path ./.models/bge-base-zh-v1.5 \
    --cache_dir ./.cache/model \
    --train_data ./Tutorials/zh/7_Fine-tuning/ft_data/training.json \
    --cache_path ./.cache/data \
    --train_group_size 8 \
    --query_max_len 512 \
    --passage_max_len 512 \
    --pad_to_multiple_of 8 \
    --query_instruction_for_retrieval 'Represent this sentence for searching relevant passages: ' \
    --query_instruction_format '{}{}' \
    --knowledge_distillation False \
	--output_dir ./test_encoder_only_base_bge-large-en-v1.5 \
    --overwrite_output_dir \
    --learning_rate 1e-5 \
    --use_mps_device
    --num_train_epochs 2 \
    --per_device_train_batch_size 2 \
    --dataloader_drop_last True \
    --warmup_ratio 0.1 \
    --gradient_checkpointing \
    --deepspeed ./examples/finetune/ds_stage0.json \
    --logging_steps 1 \
    --save_steps 1000 \
    --negatives_cross_device \
    --temperature 0.02 \
    --sentence_pooling_method cls \
    --normalize_embeddings True \
    --kd_loss_type kl_div
```

> 无法使用GPU，性能低下！

In [ ]:
# CUDA环境变量设置
%%bash
torchrun --nproc_per_node 2 \
	-m FlagEmbedding.finetune.embedder.encoder_only.base \
	--model_name_or_path BAAI/bge-large-en-v1.5 \
    --cache_dir ./cache/model \
    --train_data ./ft_data/training.json \
    --cache_path ./cache/data \
    --train_group_size 8 \
    --query_max_len 512 \
    --passage_max_len 512 \
    --pad_to_multiple_of 8 \
    --query_instruction_for_retrieval 'Represent this sentence for searching relevant passages: ' \
    --query_instruction_format '{}{}' \
    --knowledge_distillation False \
	--output_dir ./test_encoder_only_base_bge-large-en-v1.5 \
    --overwrite_output_dir \
    --learning_rate 1e-5 \
    --fp16 \
    --num_train_epochs 2 \
    --per_device_train_batch_size 2 \
    --dataloader_drop_last True \
    --warmup_ratio 0.1 \
    --gradient_checkpointing \
    --deepspeed config/ds_stage0.json \
    --logging_steps 1 \
    --save_steps 1000 \
    --negatives_cross_device \
    --temperature 0.02 \
    --sentence_pooling_method cls \
    --normalize_embeddings True \
    --kd_loss_type kl_div